<img src="./images/LIS_logo-FINAL.png" style="width:100px; float: right"></img>
<img src="./images/nasa-logo-web-rgb.png" style="width:250px; float: right"></img>

# Visualizing and Comparing LIS Output

## LIS Output Primer

LIS writes model state variables to disk at a frequency selected by the user (e.g., 6-hourly, daily, monthly). The LIS output we will be exploring was originally generated as *daily* NetCDF files, meaning one NetCDF was written per simulated day. We have converted these NetCDF files into a [Zarr](https://zarr.readthedocs.io/en/stable/) store for improved performance in the cloud.

## Import Libraries

In [ ]:
# interface to Amazon S3 filesystem
import s3fs

# interact with n-d arrays
import numpy as np
import xarray as xr

# interact with tabluar data (incl. spatial)
import pandas as pd
import geopandas as gpd

# interactive plots
import holoviews as hv
import geoviews as gv
import hvplot.pandas
import hvplot.xarray

# used to find nearest grid cell to a given location
from scipy.spatial import distance

# set bokeh as the holoviews plotting backend
hv.extension('bokeh')

## Load the LIS Output

The `xarray` library makes working with labelled n-dimensional arrays easy and efficient. If you're familiar with the `pandas` library it should feel pretty familiar.

Here we load the LIS output into an `xarray.Dataset` object:

In [ ]:
# create S3 filesystem object
s3 = s3fs.S3FileSystem(anon=False)

# define the name of our S3 bucket
bucket_name = 'eis-dh-hydro'

# define path to store on S3
lis_output_s3_path = f's3://{bucket_name}/SNOWEX-HACKWEEK/DA_SNODAS/SURFACEMODEL/LIS_HIST.d01.zarr/'

# create key-value mapper for S3 object (required to read data stored on S3)
lis_output_mapper = s3.get_mapper(lis_output_s3_path)

# open the dataset
lis_output_ds = xr.open_zarr(lis_output_mapper, consolidated=True)

# drop some unneeded variables
lis_output_ds = lis_output_ds.drop_vars(['_history', '_eis_source_path'])

## Explore the Data

Display an interactive widget for inspecting the dataset by running a cell containing the variable name. Expand the dropdown menus and click on the document and database icons to inspect the variables and attributes.

In [ ]:
lis_output_ds

### Accessing Attributes

Dataset attributes (metadata) are accessible via the `attrs` attribute:

In [ ]:
lis_output_ds.attrs

### Accessing Variables

Variables can be accessed using either **dot notation** or **square bracket notation**:

In [ ]:
# dot notation
lis_output_ds.SnowDepth_tavg

In [ ]:
# square bracket notation
lis_output_ds['SnowDepth_tavg']

#### Which syntax should I use?
While both syntaxes perform the same function, the square-bracket syntax is useful when interacting with a dataset programmatically. For example, we can define a variable `varname` that stores the name of the variable in the dataset we want to access and then use that with the square-brackets notation:

In [ ]:
varname = 'SnowDepth_tavg'

lis_output_ds[varname]

The dot notation syntax will not work this way because `xarray` tries to find a variable in the dataset named `varname` instead of the the value of the `varname` variable. When `xarray` can't find this variable, it throws an error:

In [ ]:
# uncomment and run the code below to see the error

# varname = 'SnowDepth_tavg'

# lis_output_ds.varname

### Dimensions and Coordinate Variables

The dimensions and coordinate variable fields put the "*labelled*" in "labelled n-dimensional arrays":

* **Dimensions:** labels for each dimension in the dataset (e.g., `time`)
* **Coordinates:** labels for indexing along dimensions (e.g., `'2019-01-01'`)

We can use these labels to select, slice, and aggregate the dataset.

#### Selecting/Subsetting

`xarray` provides two methods for selecting or subsetting along coordinate variables:

* index selection: `ds.isel(time=0)`
* value selection `ds.sel(time='2019-01-01')`

For example, we can select the first timestep from our dataset using index selection by passing the dimension name as a keyword argument:

In [ ]:
# remember: python indexes start at 0
lis_output_ds.isel(time=0)

Or we can use value selection to select based on the coordinate(s) (think "labels") of a given dimension:

In [ ]:
lis_output_ds.sel(time='2018-01-01')

The `.sel()` approach also allows the use of shortcuts in some cases. For example, here we select all timesteps in the month of January 2018:

In [ ]:
lis_output_ds.sel(time='2018-01')

Select a custom range of dates using Python's built-in `slice()` method:

In [ ]:
lis_output_ds.sel(time=slice('2018-01-01', '2018-01-15'))

#### Latitude and Longitude

You may have noticed that latitude (`lat`) and longitude (`lon`) are listed as data variables, not coordinate variables. This dataset would be easier to work with if `lat` and `lon` were coordinate variables and dimensions. Here we define a helper function that reads the spatial information from the dataset attributes, generates arrays containing the `lat` and `lon` values, and appends them to the dataset:

In [ ]:
def add_latlon_coords(dataset: xr.Dataset)->xr.Dataset:
    """Adds lat/lon as dimensions and coordinates to an xarray.Dataset object."""
    
    # get attributes from dataset
    attrs = dataset.attrs
    
    # get x, y resolutions
    dx = round(float(attrs['DX']), 3)
    dy = round(float(attrs['DY']), 3)
    
    # get grid cells in x, y dimensions
    ew_len = len(dataset['east_west'])
    ns_len = len(dataset['north_south'])
    
    # get lower-left lat and lon
    ll_lat = round(float(attrs['SOUTH_WEST_CORNER_LAT']), 3)
    ll_lon = round(float(attrs['SOUTH_WEST_CORNER_LON']), 3)
    
    # calculate upper-right lat and lon
    ur_lat =  ll_lat + (dy * ns_len)
    ur_lon = ll_lon + (dx * ew_len)
    
    # define the new coordinates
    coords = {
        # create an arrays containing the lat/lon at each gridcell
        'lat': np.linspace(ll_lat, ur_lat, ns_len, dtype=np.float32, endpoint=False),
        'lon': np.linspace(ll_lon, ur_lon, ew_len, dtype=np.float32, endpoint=False)
    }
    
    lon_attrs = dataset.lon.attrs
    lat_attrs = dataset.lat.attrs
    
    # rename the original lat and lon variables
    dataset = dataset.rename({'lon':'orig_lon', 'lat':'orig_lat'})
    # rename the grid dimensions to lat and lon
    dataset = dataset.rename({'north_south': 'lat', 'east_west': 'lon'})
    # assign the coords above as coordinates
    dataset = dataset.assign_coords(coords)
    dataset.lon.attrs = lon_attrs
    dataset.lat.attrs = lat_attrs
    
    return dataset

Now that the function is defined, let's use it to append `lat` and `lon` coordinates to the LIS output:

In [ ]:
lis_output_ds = add_latlon_coords(lis_output_ds)

Inspect the dataset:

In [ ]:
lis_output_ds

Now `lat` and `lon` are listed as coordinate variables and have replaced the `north_south` and `east_west` dimensions. This will make it easier to spatially subset the dataset!

#### Basic Spatial Subsetting

We can use the `slice()` function we used above on the `lat` and `lon` dimensions to select data between a range of latitudes and longitudes:

In [ ]:
lis_output_ds.sel(lat=slice(37, 41), lon=slice(-110, -101))

Notice how the sizes of the `lat` and `lon` dimensions have decreased.

#### Subset Across Multiple Dimensions

Select snow depth for Jan 2017 within a range of lat/lon:

In [ ]:
# define a range of dates to select
wy_2018_slice = slice('2017-10-01', '2018-09-30')
lat_slice = slice(37, 41)
lon_slice = slice(-110, -101)

# select the snow depth and subset to wy_2018_slice
snd_CO_wy2018_ds = lis_output_ds['SnowDepth_tavg'].sel(time=wy_2018_slice, lat=lat_slice, lon=lon_slice)

# inspect resulting dataset
snd_CO_wy2018_ds

### Plotting

We've imported two plotting libraries:

* `matplotlib`: static plots
* `hvplot`: interactive plots

We can make a quick `matplotlib`-based plot for the subsetted data using the `.plot()` function supplied by `xarray.Dataset` objects. For this example, we'll select one day and plot it:

In [ ]:
# simple matplotlilb plot
snd_CO_wy2018_ds.sel(time='2018-01-01').plot()

Similarly we can make an interactive plot using the `hvplot` accessor and specifying a `quadmesh` plot type:

In [ ]:
# hvplot based map
snd_CO_wy2018_ds.sel(time='2018-01-01').hvplot.quadmesh(geo=True, rasterize=True, project=True,
                                                        xlabel='lon', ylabel='lat', cmap='viridis',
                                                        tiles='EsriImagery')

Pan, zoom, and scroll around the map. Hover over the LIS data to see the data values.

If we try to plot more than one time-step `hvplot` will also provide a time-slider we can use to scrub back and forth in time:

In [ ]:
snd_CO_wy2018_ds.sel(time='2018-01').hvplot.quadmesh(geo=True, rasterize=True, project=True,
                             xlabel='lon', ylabel='lat', cmap='viridis',
                             tiles='EsriImagery')

From here on out we will stick with `hvplot` for plotting.

#### Timeseries Plots

We can generate a timeseries for a given grid cell by selecting and calling the plot function:

In [ ]:
# define point to take timeseries (note: must be present in coordinates of dataset)
ts_lon, ts_lat = (-106.65, 40.25)

# plot timeseries (hvplot knows how to plot based on dataset's dimensionality!)
snd_CO_wy2018_ds.sel(lat=ts_lat, lon=ts_lon).hvplot(title=f'Snow Depth Timeseries @ Lon: {ts_lon}, Lat: {ts_lat}')

In the next section we'll learn how to create a timeseries over a broader area.

## Aggregation

We can perform aggregation operations on the dataset such as `min()`, `max()`, `mean()`, and `sum()` by specifing the dimensions along which to perform the calculation.

For example we can take calculate the mean and maximum snow depth at each grid cell over water year 2018 as follows:

In [ ]:
# calculate the mean at each grid cell over the time dimension
mean_snd_CO_wy2018_ds = snd_CO_wy2018_ds.mean(dim='time')
max_snd_CO_wy2018_ds = snd_CO_wy2018_ds.max(dim='time')

# plot the mean and max snow depth
mean_snd_CO_wy2018_ds.hvplot.quadmesh(geo=True, rasterize=True, project=True,
                                   xlabel='lon', ylabel='lat', cmap='viridis',
                                   tiles='EsriImagery', title='Mean Snow Depth - WY2018') + \
    max_snd_CO_wy2018_ds.hvplot.quadmesh(geo=True, rasterize=True, project=True,
                                   xlabel='lon', ylabel='lat', cmap='viridis',
                                   tiles='EsriImagery', title='Max Snow Depth - WY2018')

### Area Average

In [ ]:
# take mean over area at each timestep
mean_snd_CO_wy2018_ds = snd_CO_wy2018_ds.mean(['lat', 'lon'])

# inspect the dataset
mean_snd_CO_wy2018_ds

In [ ]:
# plot timeseries (hvplot knows how to plot based on dataset's dimensionality!)
mean_snd_CO_wy2018_ds.hvplot()

## Comparing LIS Output

Now that we're familiar with the LIS output, let's compare it to other datasets.

First, we're going to define a helper function to get the coordinates of the grid cell nearest to a given lat/lon pair:

In [ ]:
def nearest_grid(ds, pt):
    
    """
    Returns the nearest lon and lat to pt in a given Dataset (ds).
    
    pt : input point, tuple (longitude, latitude)
    output:
        lon, lat
    """
    
    df_loc = ds[['orig_lon', 'orig_lat']].isel(time=0).to_dataframe().reset_index()
    loc_valid = df_loc.dropna()
    pts = loc_valid[['lon', 'lat']].to_numpy()
    idx = distance.cdist([pt], pts).argmin()
    
    return loc_valid['lon'].iloc[idx], loc_valid['lat'].iloc[idx]

### LIS (raster) vs. SNODAS (raster)

**TODO**

### LIS (raster) vs. SNOTEL (point)

**TODO**

# Exercise

**TODO: Simple exercise attendees should be able to complete based on tutorial. Store solution in separate notebook for reference?**